In [1]:
from pyscipopt import Model
import numpy as np

# Practica 1

## Ejercicio 1

Indices
- Cantidad de porciones: con i perteneciente a [1, 2, 3]
- Cantidad de vitaminas: con j perteneciente a [A, C, D]

Variables
- A_i - Cantidad de porciones de 30g del alimento i 

Datos
- V_ij - Cantidad de vitamina j en una porcion de 30 gramos del alimento i
- C_i - Costo de 30 gramos del alimento i
- RDM_j - racion diaria minima de vitamina j

FO
- MIN(COSTO = SUM(A_i * C_I)) - minimizar costo de la porcion

s.a
- SUM_i(A_i) >= 225/30  - la porcion tiene que tener al menos 225 grm
- SUM_i(A_I * V_ij) >= RDM_j for all j - la porcion tiene que tener todas las vitaminas minimas

In [13]:
# Vitamin content per 30g [rows: A, C, D; columns: Alimentos 1, 2, 3]
vitamin_content = np.array([
    [0.03, 0.02, 0.04],   # Vitamina A
    [0.01, 0.015, 0.005], # Vitamina C
    [0.04, 0.03, 0.02]    # Vitamina D
])

# Ración Diaria Mínima (RDM) for vitamins A, C, D
rdm = np.array([0.3, 0.12, 0.21])

# Cost per 30g portion for each food
cost = np.array([0.15, 0.10, 0.12])

# Portion size per unit and total minimum required in grams
portion_size = 30
min_total_grams = 225
min_total_portions = min_total_grams / portion_size  # 7.5 portions minimum




model = Model('Ej1')

food_portion = np.empty(len(cost), dtype='object')
for i in range(len(cost)):
  food_portion[i] = model.addVar(f'x_{i}', vtype='I')

model.setObjective(sum(food_portion[i] * cost[i] for i in range(len(cost))), sense='minimize')

model.addCons(sum(food_portion[i] for i in range(len(cost))) >= min_total_portions)

for j in range(len(rdm)):
    model.addCons(sum(food_portion[i] * vitamin_content[j, i] for i in range(len(cost))) >= rdm[j])

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej1.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 3.700000e+04
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 3 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 6 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 6 tightened bounds, 0 added holes, 1 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3 variables (0 bin, 3 int, 0 impl, 0 cont) and 4 constraints
      4 constraints of type <linear>
transformed objective value is always integral (scale: 0.01)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |va

In [23]:
print(f"costo final: {sum(sol[food_portion[i]] * cost[i] for i in range(len(food_portion)))}")
print(f"req: {225/30}; cantidad porciones: {sum(sol[food_portion[i]] for i in range(len(food_portion)))}")

for j in range(len(rdm)):
    print(f"req: {rdm[j]}; vitamina {j}: {sum(sol[food_portion[i]] * vitamin_content[j,i] for i in range(len(food_portion)))}")

costo final: 1.1800000000000002
req: 7.5; cantidad porciones: 11.0
req: 0.3; vitamina 0: 0.30000000000000004
req: 0.12; vitamina 1: 0.125
req: 0.21; vitamina 2: 0.29


## Ejercicio 2

Indices
- Cantidad de proyectos: con i perteneciente a [1, .., 6]

Variables
- A_i - Cantidad de millones de pesos invertidos en el proyecto i

Datos
- c_i - categoria de proyecto i
- U_i - utilidad del proyecto i
- fm_i - financiamiento maximo del proyecto i

FO
- MAX(utilidad = SUM(A_i * U_i)) - maximizar utilidad

s.a
- SUM_i(A_i) <= 1000 - el presupuesto total debe ser menor al disponible
- A_i <= fm_i - el financiamiento de cada proyecto no puede exceder el maximo
- A_i >= 50/100 * fm_i for all i in nuclear - el financiamiento de los proyectos nucleares tiene que ser al menos del 50%
- SUM_i(A_i) >= 300 for all i in solar - el financiamiento de proyectos nucleares tiene que ser al menos de 300 M

In [31]:
# Net utility per peso invested for each project
utility = np.array([4.4, 3.8, 4.1, 3.5, 5.1, 3.2])

# Required total financing for each project (in millions)
max_financing = np.array([220, 180, 250, 150, 400, 120])

# Total available budget (in millions)
budget = 1000

# Indices for project categories
solar_index = [0, 1]     # Projects 1 and 2
nuclear_index = [4]          # Project 5



model = Model('Ej2')

project_range = range(len(utility))
project_cash = np.empty(len(utility), dtype='object')
for i in project_range:
  project_cash[i] = model.addVar(f'x_{i}', vtype='I')

model.setObjective(sum(project_cash[i] * utility[i] for i in project_range), sense='maximize')

model.addCons(sum(project_cash[i] for i in project_range) <= 1000)

for i in project_range:
    model.addCons(project_cash[i] <= max_financing[i])
    if i in nuclear_index:
        model.addCons(project_cash[i] >= 50/100 * max_financing[i])

model.addCons(sum(project_cash[i] for i in project_range if i in solar_index) >= 300)

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej2.sol')

presolving:
(round 1, fast)       0 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, exhaustive) 0 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 1 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
(round 4, fast)       1 del vars, 8 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (5 rounds: 5 fast, 3 medium, 3 exhaustive):
 1 deleted vars, 8 deleted constraints, 0 added constraints, 15 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 5 variables (0 bin, 5 int, 0 impl, 0 cont) and 1 constraints
   

In [32]:
print(f"utilidad final: {sum(sol[project_cash[i]] * utility[i] for i in project_range)}")
print(f"req: {1000}; budget gastado: {sum(sol[project_cash[i]] for i in project_range)}")
print(f"req: {0.5 * sum(max_financing[i] for i in nuclear_index)}; financing nuclear: {sum(sol[project_cash[i]] for i in nuclear_index)}")
print(f"req: {300}; financing solar: {sum(sol[project_cash[i]] for i in solar_index)}")

utilidad final: 4527.0
req: 1000; budget gastado: 1000.0
req: 200.0; financing nuclear: 400.0
req: 300; financing solar: 350.0


## Ejercicio 3

Indices
- Cantidad de paises: con i perteneciente a [1, .., N]
- Cantidad de colores: con k perteneciente a [1, .., inf a priori, por teo de 4 colores sabemos q son 4]

Variables
- C_k: uso el color i
- P_ik: pinto el pais i del color k

Datos
- L_ij - flag que dice si el pais i es limitrofe al pais j (matriz simetrica)

FO
- MIN(cantidad_de_colores_usados = SUM_K(C_k)) - minimizar la cantidad de colores usados

s.a
- eps * SUM_i(P_ik) <= C_k for all k - si pinto algun pais del color k, C_k tiene que valer 1, aca el eps puede vale N
- SUM_k(P_ik) = 1 for all i - tengo que pintar todos los paises de un solo color
- P_ik + P_jk <= 1 for all (i,j) tal que son limitrofes for all k - no puedo pintar limitrofes del mismo color

## Ejercicio 4

Indices
- Cantidad de ubicaciones: con i perteneciente a [1, .., 3]
- Cantidad de productos: con j perteneciente a [1, .., 3]

Variables
- C_i: construyo la fabrica en la ubicacion i ∈ {0, 1}
- P_ij: cantidad de productos j fabricados en la ubicacion i ∈ Z

Datos
- CON_j: contaminacion x unidad de producto j
- B_ij: beneficio unitario de la unidad j en el sitio i
- CM_i: capacidad maxima de producion de productos del sitio i
- VCM_i: volumen de contaminacion maximo del sitio i
- PEN_i: penalizacion por unidad excedente del sitio i

FO
- MAX(Beneficio = SUM_ji(P_ji * B_ij)) - maximizar beneficio

s.a
- SUM_i(C_i) = 1: tengo que construir exactamente una fabrica
- eps * SUM_j(P_ij) <= C_i for all i: si construyo algun productos en el sitio i tengo que construir la fabri en el sitio i, en este caso espilon puede ser 1/400 que es el maximo de productos global por sitio
- SUM_j(P_ij) <= CM_i for all i: no puedo construir mas productos que el maximo del sitio
- PEN_i * (CON_GEN_i - VCM_i) <= 90000 for all i: no puedo generar mas de 90K de multas por exceso de contaminacion

In [40]:
# Contamination per unit of product (same for all locations)
contamination_per_unit = np.array([0.5, 2.0, 1.0])

# Profits per product in each location U1, U2, U3 (rows: locations, cols: products)
profits = np.array([
    [2, 5, 3],  # U1
    [4, 3, 4],  # U2
    [3, 6, 2]   # U3
])

# Maximum production capacity per location
max_production = np.array([200, 400, 300])

# Maximum allowed contamination per location
max_contamination = np.array([150, 250, 200])

# Penalty cost per cm³ of excess contamination in each location
penalty_per_cm3 = np.array([200, 150, 100])

# Maximum allowed daily penalty cost
max_total_penalty = 90000



model = Model('Ej4')

site_range = range(len(contamination_per_unit))
product_range = range(len(profits[0]))

site_construction = np.empty(len(contamination_per_unit), dtype='object')
product_production = np.empty((len(contamination_per_unit), len(profits[0])), dtype='object')
for i in site_range:
  site_construction[i] = model.addVar(f'C_{i}', vtype='B')
  for j in product_range:
    product_production[i, j] = model.addVar(f'P_{i}{j}', vtype='I')

model.setObjective(sum(product_production[i, j] * profits[i, j] for i in site_range for j in product_range), sense='maximize')

model.addCons(sum(site_construction[i] for i in site_range) == 1)
for i in site_range:
    model.addCons(1/400 * sum(product_production[i, j] for j in product_range) <= site_construction[i])
    model.addCons(sum(product_production[i, j] for j in product_range) <= max_production[i])
    model.addCons(penalty_per_cm3[i] * (sum(product_production[i, j] * contamination_per_unit[j] for j in product_range) - max_contamination[i]) <= max_total_penalty)

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej4.sol')

presolving:
(round 1, fast)       0 del vars, 1 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 2, exhaustive) 0 del vars, 1 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 3, exhaustive) 4 del vars, 1 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 4, fast)       5 del vars, 5 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 5, fast)       6 del vars, 6 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 6, fast)       7 del vars, 6 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 7, fast)       8 del vars, 7 del conss, 0 add conss, 15 chg bounds, 0 chg sides, 1 chg coeffs, 1 upgd conss, 0 impls, 0 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) 

## Ejercicio 5

\begin{align*}
\text{a) } & x \veebar y &&\Leftrightarrow\quad x + y = 1 \\[1ex]

\text{b) } & x \Rightarrow y &&\Leftrightarrow\quad x \leq y \\[1ex]

\text{c) } & x \Leftrightarrow y &&\Leftrightarrow\quad x - y = 0 \quad \text{o} \quad x = y \\[1ex]

\text{d) } & \neg(x \vee y) &&\Leftrightarrow\quad x + y = 0 \\[1ex]

\text{e) } & (x \vee y) \Rightarrow z &&\Leftrightarrow\quad x + y \leq 2z \\[1ex]

\text{f) } & x \Rightarrow (y \wedge z) &&\Leftrightarrow\quad x \leq y,\quad x \leq z \\[1ex]

\text{g) } & x \Leftrightarrow (y \wedge z) &&\Leftrightarrow\quad x \leq y,\quad x \leq z,\quad y + z \leq 1 + x \\[1ex]

\text{h) } & x \vee (y \wedge \neg z) &&\Leftrightarrow\quad \text{Usar auxiliar } a = y(1 - z):\\
& & a \leq y,\quad a \leq 1 - z,\quad a \geq y + (1 - z) - 1\\
& & x + a \geq 1 \\[1ex]

\text{i) } & x_1 \wedge x_2 \wedge \cdots \wedge x_n &&\Leftrightarrow\quad \sum_{i=1}^{n} x_i = n \\[1ex]

\text{j) } & (x_1 \wedge x_2 \wedge \cdots \wedge x_n) \Leftrightarrow y &&\Leftrightarrow\quad
y \leq x_i \ \forall i,\quad \sum_{i=1}^{n} x_i \leq n-1 + y \\[1ex]

\text{k) } & (x_1 \vee x_2 \vee \cdots \vee x_n) \Leftrightarrow y &&\Leftrightarrow\quad
x_i \leq y\ \forall i,\quad \sum_{i=1}^{n} x_i \geq y
\end{align*}


## Ejercicio 6

Indices
- Ano i: con i perteneciente a [1, .., 5]

Variables
- A_ij: Alquilo el depto del ano i al j
- Y_k: Existe un alquiler de un ano anterior a k que termina en un anio posterior a k
- Z_i: Existe un alquiler que empieza en el anio i

Datos
- B_ij: Beneficio de alquilar del ano i al j

FO
- MAX(Beneficio = SUM_ji(A_ij * B_ij)) - maximizar beneficio

s.a
- SUM_(A_ij) = 0 where (i, j) tal que i>=j: solo puedo alquilar de un ano a uno del futuro
- SUM_j(A_ij) <= 1 for all i: solo puedo alquilar hasta una unica fecha de fin en un ano

- solamente puede pasar que empiezo el alquiler un anio si no exiten alquileres de anios anteriores que termien en un anio posterior

    - SUM(A_ij) where (i,j) tal que 1 <= i <= k-1 and k + 1 <= j <= 5 >= Y_k 
    - A_ij where (i,j) tal que 1 <= i <= k-1 and k + 1 <= j <= 5 <= Y_k 
    for all 2<=k<=4: Relacion entre Y_k y los alquileres

    - SUM_j(A_ij) >= Z_k 
    - A_ij <= Z_k for all j 
    for all i: Relacion entre Z_k y los alquileres

    - Y_k + Z_k <= 1: solamente puede pasar que empiezo el alquiler un anio si no exiten alquileres de anios anteriores que termien en un anio posterior

In [51]:
for k in range(3, 2):
    print(k)

In [6]:
profits = np.array([
    [  0, 12, 22, 38, 40],   # i=0 (year 1): j=1 to 4
    [  0,  0, 13, 20, 29],   # i=1 (year 2): j=2 to 4
    [  0,  0,  0, 10, 19],   # i=2 (year 3): j=3 to 4
    [  0,  0,  0,  0, 12],   # i=3 (year 4): j=4 only (j=4 => year 5)
], dtype=int)



model = Model('Ej6')

year_range = range(len(profits))

A = np.empty((len(profits), len(profits[0])), dtype='object')
Y = np.empty(len(profits), dtype='object')
Z = np.empty(len(profits), dtype='object')

for i in year_range:
  Y[i] = model.addVar(f'Y_{i}', vtype='B')
  Z[i] = model.addVar(f'Z_{i}', vtype='B')
  for j in range(len(profits[0])):
    A[i, j] = model.addVar(f'A_{i}{j}', vtype='B')


model.setObjective(sum(A[i, j] * profits[i, j] for i in year_range for j in range(len(profits[0]))), sense='maximize')


for i in year_range:
    for j in year_range:
        if j>=i + 1:
           continue
        model.addCons(A[i, j] == 0)

for i in year_range:
    model.addCons(sum(A[i, j] for j in range(len(profits[0]))) <= 1)

for k in range(1, len(profits)):
    model.addCons(sum(A[i, j] for i in range(0, k) for j in range(k + 1, len(profits[0]))) >= Y[k])
    for i in range(0, k):
        for j in range(k + 1, len(profits[0])):
            model.addCons(A[i, j] <= Y[k])

for i in year_range:
    model.addCons(sum(A[i, j] for j in range(len(profits[0]))) >= Z[i])
    for j in range(len(profits[0])):
        model.addCons(A[i, j] <= Z[i])

for i in year_range:
    model.addCons(Y[i] + Z[i] <= 1)

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej6.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       12 del vars, 23 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 25 clqs
(round 2, fast)       13 del vars, 25 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 24 clqs
(round 3, exhaustive) 13 del vars, 25 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 30 upgd conss, 0 impls, 24 clqs
(round 4, exhaustive) 13 del vars, 51 del conss, 6 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 30 upgd conss, 0 impls, 24 clqs
(round 5, medium)     15 del vars, 52 del conss, 6 add conss, 10 chg bounds, 1 chg sides, 1 chg coeffs, 30 upgd conss, 0 impls, 21 clqs
(round 6, exhaustive) 15 del vars, 55 del conss, 9 add conss, 10 chg bounds, 1 chg sides, 1 chg coeffs, 30 upgd conss, 0 impls, 21 clqs
(round 7, exhaustive) 15 del vars, 57 del conss, 9 add conss, 10 chg bounds, 1 ch

## Ejercicio 7

Indices
- Districto i: con i perteneciente a [1, .., I]
- Colegio j: con j perteneciente a [1, .., J]

Variables
- C_j: construyo colegio en el sitio j
- S_ji: el colegio j sirve al districto i 
- DMAX: ariable auxiliar a minimizar para minmax

Datos
- d_ij: distancia del districto i al colegio j
- COS_j: costo de fijo de construir colegio en el sitio j
- T_j: capacidad maxima de un colegio en el sitio j

FO
- MIN(DMAX) - minimizar distancia maxima

s.a
- C_j <= SUM_i(S_ji) for all j: si construyo un colegio en el sitio j, tiene que servir a por lo menos 1 colegio
- S_ji <= C_j for all j for all i: si el colegio j sirve al colegio i entonces tengo que construir el colegio j
- SUM_j(S_ji) <= 1 for all i: cada districto puede ser servido como mucho por un colegio
- SUM_i(S_ji) <= 2 for all j: cada colegio puede servir a lo maximo 2 districtos
- SUM_i(S_ji * p_i) <= T_j for all j: la capacidad maxima debe ser mayor igual a los districtos que sirve
- SUM_j(C_j * COS_j) + SUM_i(P_i) * F <= B: no me puedo pasar del presupuesto total
- SUM_j(S_ju + S_jv) <= 1 for all j: los districtos u y v tienen que ser atendidos por 2 colegios distintos
- d_ij * S_ji <= DMAX for all ij: variable auxiliar a minimizar para minmax

## Ejercicio 9

Indices
- filas del cuadrado extendido: i pertenece a [1, .., 8]
- columnas del cuadrado extendido: j pertenece a [1, .., 9]

Variables
- A_ij: Coloco un alfil en la casilla ij

FO
- MAX(SUM_ij(A_ij)) - Maximixar la cantidad de alfiles en el tablero

s.a
- SUM_i(A_ij) <= 2 for all (i,j) dentro del cuadrado original: solo puede haber 2 alfiles en cada columna del cuadrado original
- SUM_j(A_ij) <= 2 for all (i,j) dentro del cuadrado original: solo puede haber 2 alfiles en cada fila del cuadrado original
- A_ij = 0 for all (i,j) fuera del cuadrado original: devolver alfiles validos

In [ ]:
model = Model('Ej9')

row_n = 17
col_n = 8

A = np.empty((row_n, col_n), dtype='object')

for i in range(row_n):
  for j in range(col_n):
    A[i, j] = model.addVar(f'A_{i}_{j}', vtype='B')

model.setObjective(sum(A[i, j] for i in range(row_n) for j in range(col_n)), sense='maximize')

for i in range(row_n):
    k = min(i, 16 - i)
    if k == 0:
        continue
    if i % 2 == 0:
        model.addCons(sum(A[i, j] for j in range((col_n - k)//2, (col_n + k)//2)) <= 2)
    else:
        model.addCons(sum(A[i, j] for j in range((col_n - k - 1)//2, (col_n + k - 1)//2)) <= 2)

for j in range(col_n):
    model.addCons(sum(A[2*i, j] for i in range(((row_n + 1) // 2))) <= 2)
    model.addCons(sum(A[2*i + 1, j] for i in range(((row_n - 1) // 2))) <= 2)

for i in range(row_n):
    for j in range(col_n):
        k = min(i, 16 - i)
        if i % 2 == 0:
            if j >= (col_n - k)//2 and j <= (col_n + k - 2)//2:
                continue
        else:
            if j >= (col_n - k - 1)//2 and j <= (col_n + k - 3)//2:
                continue
        model.addCons(A[i, j] == 0)
       
model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej9.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       72 del vars, 76 del conss, 0 add conss, 72 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       72 del vars, 81 del conss, 0 add conss, 72 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 72 del vars, 81 del conss, 0 add conss, 72 chg bounds, 0 chg sides, 0 chg coeffs, 22 upgd conss, 0 impls, 0 clqs
   (0.0s) probing: 51/64 (79.7%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (0.0s) probing aborted: 50/50 successive totally useless probings
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) symmetry computation finished: 17 generators found (max: 1500, log10 of symmetry group size: 5.12) (symcode time: 0.00)
dynamic symmetry handling statistics:
   orbitopal reduction:       no components
   orbital re

## Ejercicio 10

Variables
- x
- y
- a_1 y a_2: variables de disyuncion de x!=y

FO
- MIN(f(x,y)) - minimizar f

s.a
- x - y >= 1 - 3N(1 - a_1)
- x - y <= 1 + 3N(1 - a_2)
- x <= N
- x >= -N
- y <= N
- y >= -N

## Ejercicio 11

Indices
- Camiones: i pertenece [1, .., 5]
- sitios de input: j pertenece [A, .. , C]
- sitios de Output: k pertenece [1, .. , 3]

Variables
- A_ijk: El camion i envia A_ijk kilos de arena del sitio j al sitio k
- U_ijk: Uso el camion i para llevar arena del sitio j al sitio k

Datos
- C_jk: costo de mover un kilo del sitio j al sitio k
- D_j: disponibilidad de arena en el sitio j
- R_k: Requisito de arena en el sitio k

FO
- MIN(SUM_i(U_i * 50) + U_5 * 65 + SUM_ijk(A_ijk * C_jk)) - Minimizar el costo de mover la arena

s.a
- SUM_ik(A_ijk) <= D_j for all j: todos los sitios suministran como maximo su cantidad de arena
- SUM_ij(A_ijk) >= R_k for all k: todos los sitios reciben como minimo su requisito de arena
- SUM_kj(A_ijk) <= 600 for all i: todos los camiones pueden llevar a lo sumo 600Kg de arena
- SUM_jk(U_ijk) <= 1 for all i: cada camion solo puede ir de un sitio a otro
- SUM_i(U_iA2 + U_iB2) <= 1: el sitio 2 no puede recibir arena de A y de B al mismo tiempo
- A_ijk <= U_ijk * 600 for all ijk: si llevo arena con el camion i tengo que usar el camion i 

In [8]:
model = Model('Ej11')

# Supply at source sites A, B, C
supply = np.array([700, 600, 400])  # in kg

# Demand at destination sites 1, 2, 3
demand = np.array([800, 500, 400])  # in kg

# Cost matrix (rows: A, B, C; columns: 1, 2, 3)
cost_matrix = np.array([
    [9, 6, 5],  # From A
    [7, 4, 9],  # From B
    [4, 6, 3]   # From C
])  # cost per kg

# Truck parameters
truck_capacity = 600  # kg
regular_truck_cost = 50  # dollars
extra_truck_cost = 65    # dollars
regular_trucks = 4
max_trucks = 5

A = np.empty((max_trucks, len(supply), len(demand)), dtype='object')
U = np.empty((max_trucks, len(supply), len(demand)), dtype='object')

range_reg_trucks = range(max_trucks - 1)
range_max_trucks = range(max_trucks)
range_supply = range(len(supply))
range_demand = range(len(demand))

for i in range_max_trucks:
  for j in range_supply:
    for k in range_demand:
        A[i, j, k] = model.addVar(f'A_{i}_{j}_{k}', vtype='I')
        U[i, j, k] = model.addVar(f'U_{i}_{j}_{k}', vtype='B')


model.setObjective(
    sum(U[i, j, k] * regular_truck_cost for i in range_reg_trucks for j in range_supply for k in range_demand) \
    + sum(U[4, j, k] * extra_truck_cost for j in range_supply for k in range_demand) \
    + sum(A[i, j, k] * cost_matrix[j, k] for i in range_max_trucks for j in range_supply for k in range_demand),
    sense='minimize'
)

for j in range_supply:
    model.addCons(sum(A[i, j, k] for i in range_max_trucks for k in range_demand) <= supply[j])

for k in range_demand:
    model.addCons(sum(A[i, j, k] for i in range_max_trucks for j in range_supply) >= demand[k])

for i in range_max_trucks:
    model.addCons(sum(A[i, j, k] for j in range_supply for k in range_demand) <= truck_capacity)
    model.addCons(sum(U[i, j, k] for j in range_supply for k in range_demand) <= 1)

model.addCons(sum(U[i, 0, 1] + U[i, 1, 1] for i in range_max_trucks) <= 1)

for i in range_max_trucks:
  for j in range_supply:
    for k in range_demand:
        model.addCons(A[i, j, k] <= U[i, j, k] * truck_capacity)
       
model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej11.sol')

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 60 chg bounds, 0 chg sides, 15 chg coeffs, 0 upgd conss, 0 impls, 6 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 80 chg bounds, 0 chg sides, 15 chg coeffs, 0 upgd conss, 0 impls, 6 clqs
(round 3, fast)       0 del vars, 0 del conss, 0 add conss, 80 chg bounds, 0 chg sides, 35 chg coeffs, 0 upgd conss, 0 impls, 6 clqs
(round 4, exhaustive) 0 del vars, 0 del conss, 0 add conss, 80 chg bounds, 0 chg sides, 35 chg coeffs, 16 upgd conss, 0 impls, 6 clqs
(round 5, exhaustive) 0 del vars, 0 del conss, 0 add conss, 80 chg bounds, 0 chg sides, 35 chg coeffs, 51 upgd conss, 10 impls, 6 clqs
   (0.0s) probing cycle finished: starting next cycle
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) symmetry computation finished: 3 generators found (max: 1500, log10 of symmetry group size: 1.38) (symcode time: 0.00)
dynamic symmetry handling statist

## Ejercicio 12

Indices
- minas: i pertenece [1, .. , 4]
- Anos: j pertenece [1, .., 5]

Variables
- A_ij: La mina i esta abierta en el ano j
- O_ij: La mina i esta operando en el ano j

Datos
- R_i: regalia de la mina i en un ano
- P_i: produccion de la mina i en un ano
- C_i: calidad del los minerales de la mina i
- CM_j: calidad minima del hierro en el ano j
- V: precio de venta de una tonelada de hierro

FO
- MAX(SUM_ij(O_ij * P_i - A_ij * R_i)) - Maximizar las ganancias

s.a
- SUM_i(A_ij) <= 3 for all j: para cada ano solo puede haber 3 minas activas a lo sumo
- A_ij >= A_ij+1 for all ij: solo puedo cerrar una mina una vez
- A_i1 = 1 for all i: arrancan todas las minas abiertas
- A_ij <= O_ij for all ij: si una mina esta operando entonces tiene que estar abierta
- SUM_ij(O_ij * P_ij * C_i) >= CM_j * SUM_ij(O_ij * P_ij): el promedio total en un ano tiene que ser mayor al pedido

In [20]:
model = Model('Ej12')

# Years of operation
years = 5

# Number of mines
num_mines = 4

# Royalties per mine (in millions of dollars per year)
royalties = np.array([5, 7, 4, 6])  # shape: (4,)

# Extraction limits per mine per year (in millions of tons)
extraction_limits = np.array([2, 2.5, 1.7, 3])  # shape: (4,)

# Quality indicator per mine
qualities = np.array([1.0, 0.7, 1.5, 0.5])  # shape: (4,)

# Minimum required average quality per year
min_quality_per_year = np.array([0.9, 0.8, 1.2, 1.0, 1.1])  # shape: (5,)

# Maximum number of mines that can be exploited per year
max_mines_per_year = 3

# Iron selling price (per million tons)
selling_price = 10  # in millions of dollars

A = np.empty((num_mines, years), dtype='object')
O = np.empty((num_mines, years), dtype='object')

range_mines = range(num_mines)
range_years = range(years)

for i in range_mines:
  for j in range_years:
    A[i, j] = model.addVar(f'A_{i}_{j}', vtype='B')
    O[i, j] = model.addVar(f'O_{i}_{j}', vtype='B')


model.setObjective(
    sum(O[i, j] * extraction_limits[i] * selling_price - A[i, j] * royalties[i] for i in range_mines for j in range_years),
    sense='maximize'
)

for j in range_years:
    model.addCons(sum(O[i, j] for i in range_mines) <= max_mines_per_year)

for i in range_mines:
    for j in range_years:
        model.addCons(O[i, j] <= A[i, j])
        if j == 0:
            model.addCons(A[i, j] == 1)
        if j < years - 1:
            model.addCons(A[i, j] >= A[i, j + 1])
for j in range_years:
    model.addCons(sum(O[i, j] * extraction_limits[i] * qualities[i] for i in range_mines) >= min_quality_per_year[j] * sum(O[i, j] * extraction_limits[i] for i in range_mines))

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej12.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value -9.500000e+01
presolving:
(round 1, fast)       13 del vars, 12 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 23 clqs
(round 2, fast)       19 del vars, 30 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 17 clqs
(round 3, fast)       22 del vars, 33 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 14 clqs
(round 4, fast)       23 del vars, 34 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 13 clqs
(round 5, exhaustive) 23 del vars, 34 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 4 chg coeffs, 14 upgd conss, 0 impls, 13 clqs
(round 6, fast)       28 del vars, 42 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 4 chg coeffs, 14 upgd conss, 0 impls, 4 clqs
(round 7, fast)       30 del vars, 44 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 

## Ejercicio 13

Variables
- x
- y
- w
- M: variables minmax auxiliar

FO
- MIN(M) - minimizar el maximo abs de y + x - 2w

s.a
- x + y/2 >= 1
- 2x + w = 3
- x >= 0
- y >= 0
- w >= 0
- M >= 0
- y + x - 2w <= M
- y + x - 2w >= -M